# Use SFS_and_divergence.pk to generate dfe-alpha input file

In [76]:
import pickle

# 1. Calculate total SFS vectors

In [77]:
with open('../../data/workflow/SFS_and_divergence.pk', "rb") as f:
    df = pickle.load(f)

In [62]:
df[:5]

,Cincerta_transcript_ID,diffs0,sites0,diffs4,sites4,PAC_id,neutral_SFS,selected_SFS
0,g6935.t1,24,2777,113,699,PAC:26888200,"[609, 11, 17, 29, 6, 10, 6, 2, 8, 0, 0, 0, 0, ...","[2775, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
1,g13204.t1,28,655,38,217,PAC:26904954,"[155, 4, 0, 2, 0, 4, 3, 3, 1, 0, 0, 0, 0, 0, 0...","[519, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0..."
2,g11333.t1,231,2638,311,950,PAC:26894252,"[491, 4, 9, 4, 1, 0, 7, 2, 4, 1, 0, 0, 0, 0, 0...","[1435, 9, 2, 1, 0, 0, 4, 2, 3, 0, 0, 0, 0, 0, ..."
3,g15729.t1,450,2626,325,930,PAC:26891814,"[804, 48, 28, 18, 7, 9, 5, 3, 3, 0, 0, 0, 0, 0...","[2540, 55, 27, 11, 4, 6, 4, 7, 5, 1, 0, 0, 0, ..."
4,g5327.t1,3,1483,72,412,PAC:26901315,"[408, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...","[1478, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [63]:
def sum_SFSs(list_of_SFSs):
    # for a bunch of SFSs that are the same length sum them alllll up into one big one\
#     sfs_size = len(list_of_SFSs[0])
#     for i in range(sfs_size):
#         sum([sfs[i] for sfs in list_of_SFSs ]
    total_SFS = [sum(i) for i in zip(*list_of_SFSs)]
    return total_SFS

In [69]:
neutral = SFS(sum_SFSs(df.neutral_SFS)).fold().sfs
selected = SFS(sum_SFSs(df.selected_SFS)).fold().sfs

In [71]:
print("Neutral: " , neutral)
print("Selected: ", selected)

Neutral:  [5070211, 179500, 99584, 80066, 58587, 58433, 48633, 47268, 45738, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Selected:  [16786666, 159569, 83676, 63640, 45234, 43463, 35869, 34378, 32922, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# 2. Generate dfe-est input file
`M` = No. SFSs with different numbers of alleles sampled (m)

`MIN_ALLELE` = No. alleles sampled in SFS i (xi) [= no. elements in unfolded vector]

In [72]:
M = 1
MIN_ALLELE = len(SFS(sum_SFSs(df.neutral_SFS)).sfs)
ls = [M, MIN_ALLELE, selected, neutral]

In [74]:
with open('../../data/dfe_alpha/test/input/sfs_input_file.txt', 'w') as f:
    for i in ls[:2]: f.write(str(i)+"\n")
    for i in ls[2:4]: 
        for val in i: f.write(str(val)+" ") 
        f.write(str(val)+"\n") 
f.close()

# 3. Generate est_alpha_omega input file

In [89]:
selected = [str(i) for i in [1,sum(df.sites0),sum(df.diffs0)]]
neutral = [str(i) for i in [0, str(sum(df.sites4)), str(sum(df.diffs4))]]
print("perc Sites Diffs")
print("Selected: ", sum(df.sites0), sum(df.diffs0))
print("Neutral: ", sum(df.sites4), sum(df.diffs4))

perc Sites Diffs
Selected:  18456223 2072317
Neutral:  6224278 2019169


In [91]:
with open('../../data/dfe_alpha/test/divergence_file.txt', 'w') as f:
    f.write(" ".join(selected)+"\n") 
    f.write(" ".join(neutral))
f.close()